In [2]:
class GraphNode:
    def __init__(self, id, text):
        # node id
        # 不同 node id 可能 map 到同一个 graph id (text 相同)
        self.id = id
        self.text = text
        self.next = []
        
    def point_to(self, node):
        self.next.append(node)
        
def build_graph(graph, node, graph_id):
    graph['g_ids'][str(node.id)] = graph_id
    graph['g_ids_features'][str(node.id)] = node.text
    graph['g_adj'][str(node.id)] = []
    graph_id += 1
    if len(node.next) != 0: 
        for i in node.next:
            graph['g_adj'][str(node.id)].append(i.id)
            _, graph_id = build_graph(graph, i, graph_id)
            
    return graph, graph_id

def pad_graph(graph):
    graph_id = max(graph['g_ids'].values())
    if graph_id < 16:
        # add pad node
        for i in range(1, 16 - graph_id):
            graph['g_ids'][str(graph_id+i)] = graph_id + i
            graph['g_ids_features'][str(graph_id+i)] = 'PAD'
            graph['g_adj'][str(graph_id+i)] = []
    return graph

In [3]:
agg_str = ['NONE', 'MAXSYM', 'MINSYM', 'COUNTSYM', 'SUMSYM', 'AVGSYM']
op_str = ['EQUAL', 'MORE', 'LESS']
import json
tables = {}
with open('train.tables.jsonl', 'r') as fr:
    for line in fr:
        js = json.loads(line)
        tables[js['id']] = js
num = 0
for j in range(0, 1):
    with open('train_augment.jsonl'.format(j), 'r', encoding='utf-8') as fr:
        with open('train_augment.data'.format(j), 'w', encoding='utf-8') as fw:
            for k, line in enumerate(fr):
                js = json.loads(line)
                headers = [header.lower().replace('?', '') for header in tables[js['table_id']]['header']]
                graph = {}
                graph['seq'] = js['question'].lower().replace('?', '')
                graph['g_ids'] = {}
                graph['g_ids_features'] = {}
                graph['g_adj'] = {}
                graph['map'] = {}
                select_node = GraphNode(0, 'SELECTSYM')
                and_node = GraphNode(1, 'ANDSYM')
                and_node.point_to(select_node)

                for i in range(len(js['sql']['conds'])):
                    col_node = GraphNode(3*i+2, headers[js['sql']['conds'][i][0]] + ' WHERECOL{}'.format(i))
                    graph['map']['WHERECOL{}'.format(i)] = headers[js['sql']['conds'][i][0]]
                    op_node = GraphNode(3*i+3, op_str[js['sql']['conds'][i][1]])
                    text = 'VALUE{}'.format(i)
                    graph['map']['VALUE{}'.format(i)] = str(js['sql']['conds'][i][2]).lower().replace('?', '')
                    if 'WHERECOL{}'.format(i) in graph['seq']:
                        graph['seq'] = graph['seq'].replace(str(js['sql']['conds'][i][2]).lower().replace('?', ''), 'VALUE{}'.format(i))
                    else:
                        graph['seq'] = graph['seq'].replace(str(js['sql']['conds'][i][2]).lower().replace('?', ''), 'WHERECOL{}'.format(i) + ' VALUE{}'.format(i))
    
                    val_node = GraphNode(3*i+4, str(js['sql']['conds'][i][2]).lower().replace('?', '') + ' ' + text)
                    col_node.point_to(op_node)
                    op_node.point_to(val_node)
                    and_node.point_to(col_node)
                selcol_node = GraphNode(3*len(js['sql']['conds']) + 2, headers[js['sql']['sel']] + ' SELECTCOL')
                graph['map']['SELECTCOL'] = headers[js['sql']['sel']]

                select_node.point_to(selcol_node)
                if js['sql']['agg'] != 0:
                    agg_node = GraphNode(3*len(js['sql']['conds']) + 3, agg_str[js['sql']['agg']])
                    selcol_node.point_to(agg_node)
                graph_id = 0
                graph, _ = build_graph(graph, and_node, graph_id)
                graph = pad_graph(graph)
                fw.write(json.dumps(graph) + '\n')